# Data Preparation Notebook

This notebook focuses on the essential steps for preparing the dataset for analysis. It includes data loading, cleaning, transformation, and feature engineering processes to ensure the dataset is ready for modeling and analysis. Key tasks covered:

1. **Data Collection & Integration**
2. **Data Inspection**: 
3. **Data Cleaning** 
3. **Synthetic Data Generation**

In [1]:
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
np.random.seed(42)

## 1. Data Loading
1. **customer and customer_age dataframe**: Obtained from Shopee Code League
2. **products**: Obtained from scraped Shopee platform Data

In [3]:
customer = pd.read_csv("train.csv") # main user data file
customer_age = pd.read_csv("users.csv") # age domain etc
products = pd.read_csv("products.csv")

In [4]:
customer.columns

Index(['country_code', 'grass_date', 'user_id', 'subject_line_length',
       'last_open_day', 'last_login_day', 'last_checkout_day',
       'open_count_last_10_days', 'open_count_last_30_days',
       'open_count_last_60_days', 'login_count_last_10_days',
       'login_count_last_30_days', 'login_count_last_60_days',
       'checkout_count_last_10_days', 'checkout_count_last_30_days',
       'checkout_count_last_60_days', 'open_flag', 'row_id'],
      dtype='object')

In [5]:
customer

,country_code,grass_date,user_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id
0,4,2019-07-16 00:00:00+08:00,43,44,19,6,18,0,2,4,12,43,99,0,5,10,0,0
1,4,2019-07-16 00:00:00+08:00,102,44,9,4,8,2,9,17,18,48,90,1,1,4,1,1
2,6,2019-07-16 00:00:00+08:00,177,49,14,5,5,0,4,12,24,69,119,5,19,27,0,2
3,1,2019-07-16 00:00:00+08:00,184,49,49,9,53,0,0,1,9,23,69,1,3,6,0,3
4,6,2019-07-16 00:00:00+08:00,221,49,227,6,221,0,0,0,2,5,5,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73534,6,2019-09-02 00:00:00+08:00,127613,39,24,36,279,0,1,1,0,0,0,0,0,0,0,73534
73535,2,2019-09-02 00:00:00+08:00,127620,38,46,10,51,0,0,1,0,0,0,0,0,0,0,73535
73536,2,2019-09-02 00:00:00+08:00,127696,32,Never open,Never login,Never checkout,0,0,0,0,0,0,0,0,0,0,73536
73537,2,2019-09-02 00:00:00+08:00,127807,38,5,34,Never checkout,2,4,4,0,0,0,0,0,0,1,73537


## 2. Data Inspection
Key Task Covered:
- Analyze data types and structures
- Identify missing or incomplete data
- Detect outliers and anomalies
- Assess data distribution and patterns

### Data Inspection for Customer Table
- Left join 2 customer tables
- Filtering country to 7 - Singapore
- Dropping unnecessary columns
- Renaming columns

In [6]:
customer = pd.merge(customer, customer_age, how='left', on='user_id')

In [7]:
SG_customer = customer[customer.country_code == 7].reset_index()
SG_customer = SG_customer.drop(columns = ['index', 'country_code', 'grass_date'])
SG_customer = SG_customer.rename(columns={'user_id': 'customer_id'})

In [8]:
# Assuming customer['last_checkout_day'] contains non-integer values
SG_customer['last_checkout_day'] = pd.to_numeric(SG_customer['last_checkout_day'], errors='coerce').astype('Int64')


In [9]:
SG_customer

,customer_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,...,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id,attr_1,attr_2,attr_3,age,domain
0,299,32,19,1,2,0,1,3,34,118,...,1,2,7,0,2393,1.0,1.0,1.0,38.0,@gmail.com
1,2143,32,21,2,7,0,2,4,20,68,...,1,3,10,0,2429,1.0,1.0,2.0,26.0,@yahoo.com
2,2306,37,130,1,10,0,0,0,172,522,...,12,12,26,0,2434,1.0,1.0,4.0,56.0,@gmail.com
3,3334,32,9,13,19,1,1,2,2,10,...,0,1,3,0,2448,1.0,1.0,2.0,30.0,@gmail.com
4,5751,32,62,2,5,0,0,0,22,64,...,4,13,18,0,2486,1.0,1.0,2.0,27.0,@yahoo.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,126576,63,32,20,<NA>,0,0,1,0,0,...,0,0,0,0,69594,NaN,1.0,0.0,NaN,@yahoo.com
2334,126948,63,5,5,<NA>,1,3,3,0,0,...,0,0,0,0,69599,1.0,1.0,1.0,26.0,@gmail.com
2335,127209,63,85,889,<NA>,0,0,0,0,0,...,0,0,0,0,69607,NaN,1.0,0.0,NaN,@yahoo.com
2336,127441,63,28,474,<NA>,0,1,2,0,0,...,0,0,0,0,69611,1.0,1.0,1.0,30.0,@yahoo.com


In [10]:
SG_customer[SG_customer['last_checkout_day'] < 60]

,customer_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,...,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id,attr_1,attr_2,attr_3,age,domain
0,299,32,19,1,2,0,1,3,34,118,...,1,2,7,0,2393,1.0,1.0,1.0,38.0,@gmail.com
1,2143,32,21,2,7,0,2,4,20,68,...,1,3,10,0,2429,1.0,1.0,2.0,26.0,@yahoo.com
2,2306,37,130,1,10,0,0,0,172,522,...,12,12,26,0,2434,1.0,1.0,4.0,56.0,@gmail.com
3,3334,32,9,13,19,1,1,2,2,10,...,0,1,3,0,2448,1.0,1.0,2.0,30.0,@gmail.com
4,5751,32,62,2,5,0,0,0,22,64,...,4,13,18,0,2486,1.0,1.0,2.0,27.0,@yahoo.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,122714,63,13,1,1,0,0,0,5,12,...,1,1,1,0,69500,NaN,1.0,0.0,NaN,@gmail.com
2326,123188,63,Never open,1,1,0,0,0,19,28,...,9,12,12,0,69545,NaN,1.0,0.0,NaN,@yahoo.com
2328,123507,63,Never open,9,13,0,0,0,9,15,...,0,1,1,0,69564,NaN,1.0,0.0,NaN,@gmail.com
2331,124007,63,Never open,4,10,0,0,0,7,8,...,2,2,2,0,69587,1.0,1.0,3.0,24.0,other


In [11]:
SG_customer.head(5)

,customer_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,...,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,row_id,attr_1,attr_2,attr_3,age,domain
0,299,32,19,1,2,0,1,3,34,118,...,1,2,7,0,2393,1.0,1.0,1.0,38.0,@gmail.com
1,2143,32,21,2,7,0,2,4,20,68,...,1,3,10,0,2429,1.0,1.0,2.0,26.0,@yahoo.com
2,2306,37,130,1,10,0,0,0,172,522,...,12,12,26,0,2434,1.0,1.0,4.0,56.0,@gmail.com
3,3334,32,9,13,19,1,1,2,2,10,...,0,1,3,0,2448,1.0,1.0,2.0,30.0,@gmail.com
4,5751,32,62,2,5,0,0,0,22,64,...,4,13,18,0,2486,1.0,1.0,2.0,27.0,@yahoo.com


#### Checking For Duplicates

In [12]:
SG_customer.customer_id.value_counts()[SG_customer.customer_id.value_counts() >= 2]

customer_id
53534     2
33637     2
44714     2
25391     2
110092    2
Name: count, dtype: int64

In [13]:
SG_customer = SG_customer.drop_duplicates(subset='customer_id', keep='first')

In [14]:
SG_customer = SG_customer.reset_index()

### Data Cleaning For Products

For Products dataset, we have something scraped from Shopee Malaysia. We notice that Shopee Singapore and Malaysia have similar pricing and delivery location (since Shopee Singapore mostly imports products from other country). Here what we do:
1. Change Type for necessary columns from string to Int, Change currency from MYR to SGD based on today's exchange rate
2. Extracting Information in Specification (delivery location, etc)
3. Apply function to extract main category, middle category, and sub category
4. Dropping unecessary Columns

In [15]:
# Use regular expressions to extract 'Stock' and 'Ships From' values
products['Stock'] = products['specification'].str.extract(r'Stock\s(\d+)').fillna(0).astype(int)
products['Ships_From'] = products['specification'].str.extract(r'Ships From\s(.+)$')

# MYR to SGD
products['price_ori'] = products['price_ori']/3.21118 
products['price_actual'] = products['price_actual']/3.21118 

In [16]:
def convert_rating(value):
    try:
        if 'k' in value:
            # Convert values like '8.1k' to 8100
            return int(float(value.replace('k', '')) * 1000)
        else:
            # Convert normal string numbers to integers
            return int(value)
    except:
        return np.nan


# Apply the conversion function to the 'total_rating' column
products['total_rating'] = products['total_rating'].apply(convert_rating)
products['total_sold'] = products['total_sold'].apply(convert_rating)

In [17]:
def convert_favorite(value):
    # Remove 'Favorite (' and convert the result to string
    try:
        value = value.replace('Favorite (', '').replace(')', '')
    except:
        return np.nan
    return(convert_rating(value))

products['favorite'] = products['favorite'].apply(convert_favorite)

In [18]:
products= products.drop(['delivery', 'specification', 'w_date', 'link_ori', 'idElastic', 'sitename', 'idHash', 'id', 'pict_link', 'timestamp'], axis=1)

In [19]:
products['product_id'] = products.index + 1

In [20]:
location_to_country_non_malaysia = {
    "Mainland China": "China",
    "Indonesia": "Indonesia",
    "Korea": "South Korea",
    "Vietnam": "Vietnam",
    "Taiwan": "Taiwan",
    "Thailand": "Thailand",
    "KOTA JAKARTA SELATAN - KEBAYORAN BARU, DKI JAKARTA, ID": "Indonesia",
    "KAB. MOJOKERTO - MOJOSARI, JAWA TIMUR, ID": "Indonesia",
    "KOTA JAKARTA UTARA - KELAPA GADING, DKI JAKARTA, ID": "Indonesia",
    "KOTA JAKARTA PUSAT - TANAH ABANG, DKI JAKARTA, ID": "Indonesia",
    "KOTA JAKARTA BARAT - KEBON JERUK, DKI JAKARTA, ID": "Indonesia"
}


# Apply mapping to 'Ship_From' column
products['Ships_From'] = products['Ships_From'].apply(lambda x: location_to_country_non_malaysia.get(x, "Malaysia"))



In [21]:
products[['main_category', 'mid_category', 'sub_category']] = products['item_category_detail'].str.split('|').apply(lambda x: [x[1].strip(), x[-2].strip(), x[-1].strip()]).apply(pd.Series)

In [22]:
# Function to determine Ships_From based on main_category
def assign_ships_from(row):
    if row['main_category'] == 'Electronics':
        return 'Singapore' if np.random.rand() < 0.8 else row['Ships_From']
    else:
        return 'Singapore' if np.random.rand() < 0.4 else row['Ships_From']

# Apply function to 'Ships_From' column based on 'main_category'
products['Ships_From'] = products.apply(assign_ships_from, axis=1)


# Removing Irrelevant Columns
1. Automotive -> cars etc, as you cannot sell it in shopee singapore
2. muslim fashion
3. free gifts which cause outliers in prices

In [23]:
products = products[~((products.mid_category == 'Automotive') & (products.price_actual > 1000))]
products = products[~(products.main_category == 'Muslim Fashion')]
products = products[~(products.price_actual > 10000)]

filter_phrases = ['do not purchase', 'free gift with purchase', 'not for sale', 'free gift', 'not for sell', 'gwp', 'gift with purchase']

for i in filter_phrases:
    products = products[~(products.title.str.contains(i, case=False, na=False))]

products = products[~((products.price_actual > 1000) & (products.title.str.contains('gift', case=False, na=False)))]
products[(products.price_actual > 1000) & (products.title.str.contains('gift', case=False, na=False))]

,price_ori,item_category_detail,title,item_rating,seller_name,price_actual,total_rating,total_sold,favorite,desc,Stock,Ships_From,product_id,main_category,mid_category,sub_category


In [24]:
products.head(5)

,price_ori,item_category_detail,title,item_rating,seller_name,price_actual,total_rating,total_sold,favorite,desc,Stock,Ships_From,product_id,main_category,mid_category,sub_category
0,8.034430,Shopee | Women's Bags | Shoulder Bags | Should...,1081 PU SLING BAG KULIT leather Casual Beg Tan...,4.9,qqfwholesale,4.017215,179.0,179.0,128.0,🎉 WELCOME To QQF WHOLESALE \r\n😍 ALL item read...,2906,Singapore,1,Women's Bags,Shoulder Bags,Shoulder Bags
1,0.439091,Shopee | Home Appliances | Sewing Machines | S...,Stat Chinese Traditional Frog Buttons Knot Che...,5.0,stationery.my,0.233559,17.0,17.0,19.0,Chinese Traditional Frog Buttons Knot Cheongsa...,31701,China,2,Home Appliances,Sewing Machines,Sewing Machines and Accessories
2,2.647002,Shopee | Women's Bags | Handbags | Set Handbags,N09 READY STOCK MYFOOYIN woman handbag set 2in...,4.9,fooyin88,2.647002,8100.0,8100.0,21500.0,READY STOCK\nMEASUREMENT:\nLONG 26CM X HEIGHT ...,2044,Malaysia,3,Women's Bags,Handbags,Set Handbags
3,2.709284,Shopee | Women Clothes | Maternity Wear | Inne...,🔥Shocking Sale🔥Cotton Comfort Nursing Bra Brea...,4.9,vivi2002,2.709284,6500.0,6500.0,3400.0,PRODUCT DESCRIPTION\nMaterial:95%Cotton+5%Span...,386,Malaysia,4,Women Clothes,Maternity Wear,Innerwear
4,1245.336605,Shopee | Cameras & Drones | DSLR Cameras | DSLRs,Canon EOS 70D 18-55mm IS STM Original+64gb+ext...,5.0,xpro.gadget,852.957480,10.0,10.0,132.0,Canon 70D\nHighlights\nEF-S 18-55mm f/3.5-5.6 ...,36,Singapore,5,Cameras & Drones,DSLR Cameras,DSLRs


In [25]:
products.main_category.value_counts()

main_category
Men Clothes               2451
Health & Beauty           2408
Women Clothes             2225
Mobile & Accessories      1872
Baby & Toys               1461
Home & Living             1307
Groceries & Pets          1101
Home Appliances            936
Sports & Outdoor           787
Automotive                 778
Women's Bags               609
Watches                    530
Men's Bags & Wallets       488
Cameras & Drones           450
Women Shoes                443
Fashion Accessories        389
Computer & Accessories     384
Games, Books & Hobbies     343
Men Shoes                  209
Travel & Luggage           150
Gaming & Consoles          148
Tickets & Vouchers         103
Others                      45
Name: count, dtype: int64

In [26]:
products.isna().sum()

price_ori                200
item_category_detail       0
title                      0
item_rating               11
seller_name               11
price_actual              15
total_rating              11
total_sold                11
favorite                1037
desc                       0
Stock                      0
Ships_From                 0
product_id                 0
main_category              0
mid_category               0
sub_category               0
dtype: int64

-1 -> No rating

In [27]:
# Fill price_ori with price_actual if price_ori is NA and vice versa
products.loc[products['price_ori'].isna() & products['price_actual'].notna(), 'price_ori'] = products['price_actual']
products.loc[products['price_actual'].isna() & products['price_ori'].notna(), 'price_actual'] = products['price_ori']

# Drop rows where both price_ori and price_actual are NA
products.dropna(subset=['price_ori', 'price_actual'], how='all', inplace=True)

# Fill total_rating with 0 if NA, and item_rating with -1 if NA
products['total_rating'].fillna(0, inplace=True)
products['item_rating'].fillna(-1, inplace=True)

# Fill total_sold and favorite with 0 if NA
products['total_sold'].fillna(0, inplace=True)
products['favorite'].fillna(0, inplace=True)

/var/folders/44/pbhm_xj112ndmk4mf3bf564c0000gn/T/ipykernel_7000/138345870.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  products['total_rating'].fillna(0, inplace=True)
/var/folders/44/pbhm_xj112ndmk4mf3bf564c0000gn/T/ipykernel_7000/138345870.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always 

## 3. Synthetic Data Generation

We have shortage on data, since we only have checkout count but not order generated, we plan to make synthetic data for the orders. Our plan for making synthetic data is:
1. Make Synthetic Data for Age, then
2. Make Synthetic Data for checkout count in last 6 months and 1 year, then
3. Make Synthetic Data for orders, then
4. Match last checkout date between customer dataset and orders dataset, then
5. Match last login date with last checkout date.
6. Make Synthetic Data for prices less than 25 cents.

### Making Synthetic Data for Age
We assume age of customer are independent of one another and normally distributed accross all sample customer. Based on Statista report on cross border online shopper in Singapore, we obtain age groups follows distribution as follow:
- 18-24 : 13%
- 25-34 : 20%
- 35-44 : 22%
- 45-54 : 22%
- 55-64   : 23%

In [28]:
np.random.seed(42)

In [29]:
# Define the age ranges as tuples
age_ranges = [(18, 24), (25, 34), (35, 44), (45, 54), (55, 64)]
age_probabilities = [0.13, 0.20, 0.22, 0.22, 0.23]

# Identify missing age values
missing_age_mask = SG_customer['age'].isna()

# Randomly select indices based on probabilities, then use these indices to select the range
chosen_indices = np.random.choice(len(age_ranges), size=missing_age_mask.sum(), p=age_probabilities)
random_ages = [np.random.randint(age_ranges[i][0], age_ranges[i][1] + 1) for i in chosen_indices]

# Fill missing age values with the generated random ages
SG_customer.loc[missing_age_mask, 'age'] = random_ages

In [30]:
SG_customer['gender'] = np.random.choice(['Male', 'Female'], size=len(SG_customer))

### Making Synthetic Data for Checkout Count in last 6 Months and 1 Year
The idea behind this is to apply linear regression for each customer, as we have checkout count for last 10 days, last 30 days, and last 60 days.

In [31]:
from sklearn.linear_model import LinearRegression

In [32]:
# Prepare the data in the required format
# Create the feature array (time points: 10, 30, 60 days)
time_points = np.array([10, 30, 60]).reshape(-1, 1)

# For each customer, we will fit a linear regression model
def predict_checkout_counts(row):
    # The target array (checkout counts corresponding to time points)
    checkout_counts = np.array([
        row['checkout_count_last_10_days'], 
        row['checkout_count_last_30_days'], 
        row['checkout_count_last_60_days']
    ])
    
    # Fit the linear regression model
    model = LinearRegression()
    model.fit(time_points, checkout_counts)
    
    # Predict checkout counts for 180 days (6 months) and 365 days (1 year)
    checkout_6_months = model.predict(np.array([[180]]))[0]
    checkout_1_year = model.predict(np.array([[365]]))[0]
    
    # Add some random noise for variability
    checkout_6_months += np.random.normal(0, 5)  # Adjust the noise level as needed
    checkout_1_year += np.random.normal(0, 10)   # Adjust the noise level as needed

    checkout_6_months = max(checkout_6_months, 0)
    checkout_1_year = max(checkout_1_year, 0)

    return pd.Series([checkout_6_months, checkout_1_year])

# Apply the function to each row in the DataFrame
SG_customer[['checkout_count_last_6_months', 'checkout_count_last_1_year']] = SG_customer.apply(predict_checkout_counts, axis=1)

# Ensure that checkout_count_last_1_year is always greater than checkout_count_last_6_months
SG_customer['checkout_count_last_1_year'] = SG_customer.apply(
    lambda row: max(row['checkout_count_last_1_year'], row['checkout_count_last_6_months'] + np.random.uniform(0, 5)),
    axis=1
)

In [33]:
print(SG_customer[['checkout_count_last_10_days', 'checkout_count_last_30_days', 'checkout_count_last_60_days', 'checkout_count_last_6_months', 'checkout_count_last_1_year']].head())

   checkout_count_last_10_days  checkout_count_last_30_days  \
0                            1                            2   
1                            1                            3   
2                           12                           12   
3                            0                            1   
4                            4                           13   

   checkout_count_last_60_days  checkout_count_last_6_months  \
0                            7                     21.011692   
1                           10                     26.745057   
2                           26                     56.743174   
3                            3                      6.420100   
4                           18                     55.674291   

   checkout_count_last_1_year  
0                   32.184059  
1                   75.095482  
2                  104.340498  
3                   14.534052  
4                   99.998369  


In [34]:
# Apply rounding to ensure values are integers
SG_customer['checkout_count_last_6_months'] = SG_customer['checkout_count_last_6_months'].round().astype(int)
SG_customer['checkout_count_last_1_year'] = SG_customer['checkout_count_last_1_year'].round().astype(int)

In [35]:
SG_customer.head(5)

,index,customer_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,...,open_flag,row_id,attr_1,attr_2,attr_3,age,domain,gender,checkout_count_last_6_months,checkout_count_last_1_year
0,0,299,32,19,1,2,0,1,3,34,...,0,2393,1.0,1.0,1.0,38.0,@gmail.com,Female,21,32
1,1,2143,32,21,2,7,0,2,4,20,...,0,2429,1.0,1.0,2.0,26.0,@yahoo.com,Male,27,75
2,2,2306,37,130,1,10,0,0,0,172,...,0,2434,1.0,1.0,4.0,56.0,@gmail.com,Male,57,104
3,3,3334,32,9,13,19,1,1,2,2,...,0,2448,1.0,1.0,2.0,30.0,@gmail.com,Female,6,15
4,4,5751,32,62,2,5,0,0,0,22,...,0,2486,1.0,1.0,2.0,27.0,@yahoo.com,Female,56,100


In [36]:
sum(SG_customer.checkout_count_last_1_year)

91140

### Making Synthetic Data for Orders
For orders there are 3 main parts that need to be considered:
1. Order Time (morning/noon/night)
2. Order Date (weekdays/weekends/payday)
3. Product Categories


For Order Time, we will assume that it is independent for each customer, we will just apply random distribution with probabilities specified.

For Order Date, it is highly dependent on Checkout Count.
Order Date -> Assume all other than weekends have same probability, some public holiday should have more probability + shopee promotional on 9.9, 10.10, 11.11, 12.12, Black Friday etc. 

For Product Categories: random

In [37]:
from datetime import datetime, timedelta

In [38]:
SG_customer

,index,customer_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,...,open_flag,row_id,attr_1,attr_2,attr_3,age,domain,gender,checkout_count_last_6_months,checkout_count_last_1_year
0,0,299,32,19,1,2,0,1,3,34,...,0,2393,1.0,1.0,1.0,38.0,@gmail.com,Female,21,32
1,1,2143,32,21,2,7,0,2,4,20,...,0,2429,1.0,1.0,2.0,26.0,@yahoo.com,Male,27,75
2,2,2306,37,130,1,10,0,0,0,172,...,0,2434,1.0,1.0,4.0,56.0,@gmail.com,Male,57,104
3,3,3334,32,9,13,19,1,1,2,2,...,0,2448,1.0,1.0,2.0,30.0,@gmail.com,Female,6,15
4,4,5751,32,62,2,5,0,0,0,22,...,0,2486,1.0,1.0,2.0,27.0,@yahoo.com,Female,56,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,2333,126576,63,32,20,<NA>,0,0,1,0,...,0,69594,NaN,1.0,0.0,42.0,@yahoo.com,Female,11,14
2329,2334,126948,63,5,5,<NA>,1,3,3,0,...,0,69599,1.0,1.0,1.0,26.0,@gmail.com,Male,4,8
2330,2335,127209,63,85,889,<NA>,0,0,0,0,...,0,69607,NaN,1.0,0.0,61.0,@yahoo.com,Male,0,1
2331,2336,127441,63,28,474,<NA>,0,1,2,0,...,0,69611,1.0,1.0,1.0,30.0,@yahoo.com,Female,5,9


In [39]:
# Assume 29000 unique customers
num_customers = len(SG_customer) + 100

# Define proportions for each preference group
high_preference_ratio = 0.3  # 50% of customers prefer discounts highly
moderate_preference_ratio = 0.15  # 30% are moderately responsive
low_preference_ratio = 0.2    # 20% are indifferent
a_preference_ratio = 0.2 
b_preference_ratio = 0.15 


# Calculate group sizes
high_preference_size = int(num_customers * high_preference_ratio)
moderate_preference_size = int(num_customers * moderate_preference_ratio)
low_preference_size = int(num_customers * low_preference_ratio)
a_preference_size = int(num_customers * a_preference_ratio)
b_preference_size = num_customers - high_preference_size - moderate_preference_size - low_preference_size - a_preference_size

# Define weight arrays for each preference group
high_preference_weights = [8, 2, 1, 5]   # Strong discount preference
moderate_preference_weights = [4, 2, 1, 3]  # Moderate discount preference
low_preference_weights = [2, 2, 1, 2]    # Indifferent to discount
a_preference_weights = [2, 2, 1, 5]  # a
b_preference_weights = [6, 2, 1, 6]  # b

# Assign weights to each customer
weights = (
    [high_preference_weights] * high_preference_size +
    [moderate_preference_weights] * moderate_preference_size +
    [low_preference_weights] * low_preference_size + 
    [a_preference_weights] * a_preference_size +
    [b_preference_weights] * b_preference_size
)

# Shuffle to randomize the distribution among customers
np.random.shuffle(weights)

# Create a DataFrame to store each customer's weights
customer_weights = pd.DataFrame({
    'SG_customer': [f'SG_customer_{i}' for i in range(1, num_customers + 1)],
    'weights': weights
})


In [40]:
num_rows = sum(SG_customer.checkout_count_last_1_year)
campaign_types = ['Flash Sale', 'Seasonal Sales', 'Bundle promotions', 'Mega Sales', 'Livestream Exclusive', 'Next Day Delivery']
campaign_costs = np.random.randint(1000, 10000, size=num_rows)

# Define all special dates
high_priority_mega_sales_dates = ['2019-11-11', '2019-09-09', '2019-10-10', '2019-12-12']
additional_mega_sales_dates = [
    '2019-03-03', '2019-04-04', '2019-05-05',
    '2019-06-06', '2019-07-07', '2019-08-08'
]

other_seasonal_sales_dates = [
    datetime(2019, 2, 5),   # Chinese New Year Sale
    datetime(2019, 5, 1),   # Labor Day Sale
    datetime(2019, 6, 5),   # Hari Raya Puasa Sale (Eid al-Fitr)
    datetime(2019, 8, 9),   # Singapore National Day Sale
    datetime(2019, 8, 11),  # Hari Raya Haji Sale (Eid al-Adha)
    datetime(2019, 10, 27), # Deepavali Sale (Diwali)
    datetime(2019, 12, 2),  # Cyber Monday Sale
    datetime(2019, 12, 25), # Christmas Sale
    datetime(2019, 12, 31)  # New Year’s Eve Sale
]

high_priority_seasonal_sales_dates = ['2019-11-29']

# Combine all dates into one list

high_priority_mega_sales_dates = [datetime.strptime(date, '%Y-%m-%d') for date in high_priority_mega_sales_dates]
additional_mega_sales_dates = [datetime.strptime(date, '%Y-%m-%d') for date in additional_mega_sales_dates]
high_priority_seasonal_sales_dates = [datetime.strptime(date, '%Y-%m-%d') for date in high_priority_seasonal_sales_dates]

# Function to determine if a date is a special date
def is_more_special_date(date):
    return date in high_priority_mega_sales_dates or date in high_priority_seasonal_sales_dates 

def is_less_special_date(date):
    return date in additional_mega_sales_dates or date in other_seasonal_sales_dates

# Updated day_weight function to select weights based on date and index
def day_weight(date, index):
    weights = customer_weights.iloc[index]['weights']
    if is_more_special_date(date):
        return weights[0]  # Discount weight
    elif is_less_special_date(date):
        return weights[3]  # Discount weight
    elif date.weekday() >= 4:  # Weekends (Friday, Saturday, Sunday)
        return weights[1]  # Weekend weight
    else:
        return weights[2]  # Normal day weight


# Function to assign weights to time periods
def time_weight(hour):
    if 7 <= hour <= 9 or 18 <= hour <= 23:  # High-probability times: 8-11AM, 6-11PM
        return 2.5
    elif 11 <= hour <= 15:
        return 2.0
    else:
        return 0.5

# Generate probability distribution for dates in 2019
def generate_date_distribution(start_date, end_date, index):
    dates = pd.date_range(start_date, end_date)
    weights = np.array([day_weight(date, index) for date in dates])
    probabilities = weights / weights.sum()  # Normalize weights to sum to 1
    return dates, probabilities

# Generate probability distribution for time ranges in a day
def generate_time_distribution():
    # Define time ranges and assign weights
    time_ranges = [
        (7, 9),    # Morning
        (9, 11),
        (11, 15),  # Lunch
        (18, 23),  # Evening
        (0, 7),    # Midnight to early morning
        (15, 18)   # Afternoon
    ]
    weights = np.array([time_weight((start + end) // 2) for start, end in time_ranges])
    probabilities = weights / weights.sum()  # Normalize weights to sum to 1
    return time_ranges, probabilities

# Function to generate order times based on the weighted probability distributions
def generate_order_times(num_orders, start_date, end_date, idx):
    dates, date_probabilities = generate_date_distribution(start_date, end_date, idx)
    time_ranges, time_probabilities = generate_time_distribution()

    orders = []
    for _ in range(num_orders):
        # Choose a date based on the probability distribution
        chosen_date = np.random.choice(dates, p=date_probabilities)

        # Convert numpy datetime64 to pandas Timestamp, then to Python datetime
        chosen_date = pd.Timestamp(chosen_date).to_pydatetime()

        # Choose a time range based on the probability distribution
        chosen_time_range = np.random.choice(len(time_ranges), p=time_probabilities)
        start_hour, end_hour = time_ranges[chosen_time_range]
        
        # Randomly pick an hour and minute within the chosen time range
        chosen_hour = np.random.randint(start_hour, end_hour + 1)
        chosen_minute = np.random.randint(0, 60)
        
        # Combine the chosen date and time into a datetime object
        order_datetime = datetime(
            chosen_date.year, chosen_date.month, chosen_date.day,
            chosen_hour, chosen_minute
        )

        is_campaign = False
        additional_discount = 0
        if is_more_special_date(chosen_date) or is_less_special_date(chosen_date):
            is_campaign = True
            additional_discount = np.random.randint(4, 16) * 5

        orders.append([order_datetime, is_campaign, additional_discount])

    return orders

In [41]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [42]:

# # Function to generate random order times within a specific time range
# def generate_order_times(num_orders, start_date, end_date):
#     return [start_date + (end_date - start_date) * np.random.random() for _ in range(num_orders)]

# Function to generate orders for each customer
def generate_orders_for_customer(customer_row, product_ids, idx):
    orders = []

     # Generate order times for the last 10 days
    total_orders_last_10_days = customer_row['checkout_count_last_10_days']
    if total_orders_last_10_days > 0:
        start_10_days = datetime(2019, 12, 31) - timedelta(days=10)
        end_10_days = datetime(2019, 12, 31)
        order_times_last_10_days = generate_order_times(total_orders_last_10_days, start_10_days, end_10_days, idx)
        orders.extend([(order_time, np.random.choice(product_ids), customer_row['customer_id']) for order_time in order_times_last_10_days])

    # Generate order times for the last 30 days
    total_orders_last_30_days = customer_row['checkout_count_last_30_days'] - customer_row['checkout_count_last_10_days']
    if total_orders_last_30_days > 0:
        start_30_days = datetime(2019, 12, 31) - timedelta(days=30)
        end_30_days = datetime(2019, 12, 31) - timedelta(days=11)
        order_times_last_30_days = generate_order_times(total_orders_last_30_days, start_30_days, end_30_days, idx)
        orders.extend([(order_time, np.random.choice(product_ids), customer_row['customer_id']) for order_time in order_times_last_30_days])
    
    # Generate order times for the last 60 days (excluding last 30 days)
    total_orders_last_60_days = customer_row['checkout_count_last_60_days'] - customer_row['checkout_count_last_30_days']
    if total_orders_last_60_days > 0:
        start_60_days = datetime(2019, 12, 31) - timedelta(days=60)
        end_60_days = datetime(2019, 12, 31) - timedelta(days=31)
        order_times_last_60_days = generate_order_times(total_orders_last_60_days, start_60_days, end_60_days, idx)
        orders.extend([(order_time, np.random.choice(product_ids), customer_row['customer_id']) for order_time in order_times_last_60_days])
    
    # Generate order times for the last 180 days (excluding last 30 days)
    total_orders_last_6_months = customer_row['checkout_count_last_6_months'] - customer_row['checkout_count_last_60_days']
    if total_orders_last_6_months> 0:
        start_60_days = datetime(2019, 12, 31) - relativedelta(months=6)
        end_60_days = datetime(2019, 12, 31) - timedelta(days=61)
        order_times_last_6_months = generate_order_times(total_orders_last_6_months, start_60_days, end_60_days, idx)
        orders.extend([(order_time, np.random.choice(product_ids), customer_row['customer_id']) for order_time in order_times_last_6_months])

    # Generate order times for the rest of the year (excluding last 60 days)
    total_orders_last_1_year = customer_row['checkout_count_last_1_year'] - customer_row['checkout_count_last_6_months'] 
    if total_orders_last_1_year > 0:
        start_1_year = datetime(2019, 1, 1)
        end_1_year = datetime(2019, 12, 31) - relativedelta(months=6) - timedelta(days=1)
        order_times_last_1_year = generate_order_times(total_orders_last_1_year, start_1_year, end_1_year, idx)
        orders.extend([(order_time, np.random.choice(product_ids), customer_row['customer_id']) for order_time in order_times_last_1_year])

    # Generate order IDs and construct order records
    order_ids = [i for i in range(len(order_dataset) + 1, len(order_dataset) + len(orders) + 1)]
    
    orders_df = pd.DataFrame({
        'order_id': order_ids,
        'order_time': [order[0][0] for order in orders],
        'is_campaign': [order[0][1] for order in orders],
        'additional_discount': [order[0][2] for order in orders],
        'product_id': [order[1] for order in orders],
        'customer_id': [order[2] for order in orders]
    })

    return orders_df

# Initialize the order dataset
order_dataset = pd.DataFrame(columns=['order_id', 'order_time', 'product_id', 'customer_id'])

# Get product IDs from the product dataset
product_ids = products['product_id'].values

# Generate orders for each customer
for idx, customer_row in SG_customer.iterrows():
    customer_orders = generate_orders_for_customer(customer_row, product_ids, idx)
    order_dataset = pd.concat([order_dataset, customer_orders], ignore_index=True)


/var/folders/44/pbhm_xj112ndmk4mf3bf564c0000gn/T/ipykernel_7000/3382289806.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  order_dataset = pd.concat([order_dataset, customer_orders], ignore_index=True)
/var/folders/44/pbhm_xj112ndmk4mf3bf564c0000gn/T/ipykernel_7000/3382289806.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  order_dataset = pd.concat([order_dataset, customer_orders], ignore_index=True)
/var/folders/44/pbhm_xj112ndmk4mf3bf564c0000gn/T/ipykernel_7000/3382289806.py:72: Futu

In [43]:
print(idx)

2332


In [44]:
# Function to assign campaign names based on the order date
def get_campaign_name(order_time):
    order_date = order_time.date()
    if order_date in [d.date() for d in other_seasonal_sales_dates]:
        return "Other Seasonal Sales Date"
    elif order_date in [d.date() for d in high_priority_mega_sales_dates]:
        return "High Priority Mega Sales Date"
    elif order_date in [d.date() for d in additional_mega_sales_dates]:
        return "Additional Mega Sales Date"
    elif order_date in [d.date() for d in high_priority_seasonal_sales_dates]:
        return "High Priority Seasonal Sales Date"
    else:
        return np.nan  # If no match, return None


# Apply the function to create the 'campaign_name' column
order_dataset['campaign_name'] = order_dataset['order_time'].apply(get_campaign_name)

In [45]:
order_dataset

,order_id,order_time,product_id,customer_id,is_campaign,additional_discount,campaign_name
0,1,2019-12-25 17:01:00,16498,299,True,65.0,Other Seasonal Sales Date
1,2,2019-12-01 20:14:00,8523,299,False,0.0,NaN
2,3,2019-11-21 09:53:00,7396,299,False,0.0,NaN
3,4,2019-11-29 08:09:00,2446,299,True,40.0,High Priority Seasonal Sales Date
4,5,2019-11-08 09:11:00,9353,299,False,0.0,NaN
...,...,...,...,...,...,...,...
91340,91341,2019-08-15 10:36:00,3765,127744,False,0.0,NaN
91341,91342,2019-10-10 21:36:00,13071,127744,True,40.0,High Priority Mega Sales Date
91342,91343,2019-02-23 15:47:00,4630,127744,False,0.0,NaN
91343,91344,2019-01-05 22:41:00,16511,127744,False,0.0,NaN


In [46]:
SG_customer

,index,customer_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,...,open_flag,row_id,attr_1,attr_2,attr_3,age,domain,gender,checkout_count_last_6_months,checkout_count_last_1_year
0,0,299,32,19,1,2,0,1,3,34,...,0,2393,1.0,1.0,1.0,38.0,@gmail.com,Female,21,32
1,1,2143,32,21,2,7,0,2,4,20,...,0,2429,1.0,1.0,2.0,26.0,@yahoo.com,Male,27,75
2,2,2306,37,130,1,10,0,0,0,172,...,0,2434,1.0,1.0,4.0,56.0,@gmail.com,Male,57,104
3,3,3334,32,9,13,19,1,1,2,2,...,0,2448,1.0,1.0,2.0,30.0,@gmail.com,Female,6,15
4,4,5751,32,62,2,5,0,0,0,22,...,0,2486,1.0,1.0,2.0,27.0,@yahoo.com,Female,56,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,2333,126576,63,32,20,<NA>,0,0,1,0,...,0,69594,NaN,1.0,0.0,42.0,@yahoo.com,Female,11,14
2329,2334,126948,63,5,5,<NA>,1,3,3,0,...,0,69599,1.0,1.0,1.0,26.0,@gmail.com,Male,4,8
2330,2335,127209,63,85,889,<NA>,0,0,0,0,...,0,69607,NaN,1.0,0.0,61.0,@yahoo.com,Male,0,1
2331,2336,127441,63,28,474,<NA>,0,1,2,0,...,0,69611,1.0,1.0,1.0,30.0,@yahoo.com,Female,5,9


In [47]:
from datetime import date

In [48]:
# Ensure order_time is in datetime format
order_dataset['order_time'] = pd.to_datetime(order_dataset['order_time'], errors='coerce')

# Function to apply the logic based on 'last_checkout_day'
def process_checkout(row):
    min_order_time = order_dataset[order_dataset.customer_id == row.customer_id]['order_time'].max()
    if pd.notnull(min_order_time):
        row['last_checkout_day'] = (date(2019, 12, 31) - min_order_time.date()).days
    else:
        row['last_checkout_day'] = 'Never Checkout'  # or any default value if needed
    return row

# Apply the function to the DataFrame
SG_customer = SG_customer.apply(process_checkout, axis=1)


In [49]:
SG_customer

,index,customer_id,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,...,open_flag,row_id,attr_1,attr_2,attr_3,age,domain,gender,checkout_count_last_6_months,checkout_count_last_1_year
0,0,299,32,19,1,6,0,1,3,34,...,0,2393,1.0,1.0,1.0,38.0,@gmail.com,Female,21,32
1,1,2143,32,21,2,0,0,2,4,20,...,0,2429,1.0,1.0,2.0,26.0,@yahoo.com,Male,27,75
2,2,2306,37,130,1,0,0,0,0,172,...,0,2434,1.0,1.0,4.0,56.0,@gmail.com,Male,57,104
3,3,3334,32,9,13,19,1,1,2,2,...,0,2448,1.0,1.0,2.0,30.0,@gmail.com,Female,6,15
4,4,5751,32,62,2,3,0,0,0,22,...,0,2486,1.0,1.0,2.0,27.0,@yahoo.com,Female,56,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,2333,126576,63,32,20,62,0,0,1,0,...,0,69594,NaN,1.0,0.0,42.0,@yahoo.com,Female,11,14
2329,2334,126948,63,5,5,67,1,3,3,0,...,0,69599,1.0,1.0,1.0,26.0,@gmail.com,Male,4,8
2330,2335,127209,63,85,889,276,0,0,0,0,...,0,69607,NaN,1.0,0.0,61.0,@yahoo.com,Male,0,1
2331,2336,127441,63,28,474,67,0,1,2,0,...,0,69611,1.0,1.0,1.0,30.0,@yahoo.com,Female,5,9


In [50]:
sum(SG_customer.checkout_count_last_1_year)

91140

### Making Synthetic Data for prices less than 10 cents
Although Pricing is similar for most products (since the supplier is the same), some products which has less price less than 10 cents seems a bit suspicious. We will first identify distribution of such products and apply function to match the actual price in Singapore Marketplace

In [51]:
products[products.price_ori < 1].head()

,price_ori,item_category_detail,title,item_rating,seller_name,price_actual,total_rating,total_sold,favorite,desc,Stock,Ships_From,product_id,main_category,mid_category,sub_category
1,0.439091,Shopee | Home Appliances | Sewing Machines | S...,Stat Chinese Traditional Frog Buttons Knot Che...,5.0,stationery.my,0.233559,17.0,17.0,19.0,Chinese Traditional Frog Buttons Knot Cheongsa...,31701,China,2,Home Appliances,Sewing Machines,Sewing Machines and Accessories
7,0.152592,Shopee | Mobile & Accessories | Cases & Covers...,Mobile Phone Smartphone Powerbank Soft Pouch B...,4.9,orangetechnology,0.152592,493.0,493.0,219.0,Mobile Phone Smartphone Power bank Soft Pouch ...,4135,Malaysia,8,Mobile & Accessories,Cases & Covers,Mobile Pouches
13,0.137021,Shopee | Health & Beauty | Fragrances | Gift S...,VIRAL Perfum Mini Harga Bawah Pasaran Online H...,4.9,siowgogo,0.137021,1900.0,1900.0,1900.0,VIRAL Perfume!!!\n\nRANDOM ONLY\n\nPerfume mur...,30321,Singapore,14,Health & Beauty,Fragrances,Gift Set & Sampler
17,0.031141,Shopee | Home & Living | Stationery | Writing ...,HERMAN - (1Pcs) 0.05mm Gel Pen Ball Point Stud...,4.9,herman.my,0.031141,1700.0,1700.0,1400.0,✅ Dropship/Reseller/Wholesale are welcome!\n✅ ...,51965,Malaysia,18,Home & Living,Stationery,Writing Utensils
18,0.077853,Shopee | Home & Living | Kitchen Storage | Foo...,⚡️READY STOCK⚡️ BALANG 180ML KUIH/BALANG KUEH/...,5.0,choonkeat90,0.077853,463.0,463.0,770.0,SUITABLE FOR :\n#COOKIES\n#PEPPER\n#BISCUIT\n#...,359021,Malaysia,19,Home & Living,Kitchen Storage,Food Storage & Dispensers


**WARNING** Need more research, for now its only by observation, will be multiplied by scalar factor 4:

In [52]:
# Apply different multipliers based on the conditions
products.loc[products['price_ori'] < 0.01, ['price_ori', 'price_actual']] = products.loc[products['price_ori'] < 0.01, ['price_ori', 'price_actual']].apply(lambda x: x * np.random.randint(10, 15), axis=1)
products.loc[products['price_ori'] < 0.1, ['price_ori', 'price_actual']] = products.loc[products['price_ori'] < 0.1, ['price_ori', 'price_actual']].apply(lambda x: x * np.random.randint(8,11), axis=1)
products.loc[products['price_ori'] < 0.5, ['price_ori', 'price_actual']] = products.loc[products['price_ori'] < 0.5, ['price_ori', 'price_actual']].apply(lambda x: x * np.random.randint(1, 4), axis=1)
products.loc[products['price_ori'] < 1, ['price_ori', 'price_actual']] = products.loc[products['price_ori'] < 1, ['price_ori', 'price_actual']].apply(lambda x: x * np.random.randint(1, 3), axis=1)

In [53]:
order_dataset.to_csv('orders.csv', index=False)
SG_customer.to_csv('SG_customers.csv', index=False)
products.to_csv('products_cleaned.csv', index=False)

## Data Validation
To be Implemented